In [34]:
import pandas as pd
import random 
import pickle
import numpy as np
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.models import Sequential, load_model  # Corrected Sequential and load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop  # Corrected the spelling of tensorflow and the location of RMSprop


In [32]:
!pip install tensorflow


  Using cached tensorflow-2.16.1-cp312-cp312-macosx_12_0_arm64.whl.metadata (4.1 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.5.4-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.10.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.5 kB)
  Using cached ml_dtypes-0.3.2-cp312-cp312-macosx_10_9_universal2.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached protobuf-4.25.3-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached six-1.16.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached typing_extensions-4.10.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached wrapt-1.16.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached grpcio-1.62.1-cp312-cp31

In [14]:
text_df = pd.read_csv("voicecommands.txt")
text_df.head()

,Jarvis,Values
0,Jarvis,what time is it?
1,Jarvis,check my email.
2,Jarvis,what's in the news today?
3,Jarvis,read the document named 'Quarterly Report'.
4,Jarvis,open the file 'Presentation.ppt'.


In [26]:
text = list(text_df.Values.values)
joined_text = " ".join(text)

In [27]:
partial_text = joined_text[:10000]

In [28]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [29]:
tokens

['what',
 'time',
 'is',
 'it',
 'check',
 'my',
 'email',
 'what',
 's',
 'in',
 'the',
 'news',
 'today',
 'read',
 'the',
 'document',
 'named',
 'quarterly',
 'report',
 'open',
 'the',
 'file',
 'presentation',
 'ppt',
 'add',
 'the',
 'modified',
 'files',
 'to',
 'git',
 'push',
 'the',
 'changes',
 'to',
 'the',
 'repository',
 'what',
 's',
 'on',
 'my',
 'calendar',
 'for',
 'today',
 'play',
 'some',
 'music',
 'remind',
 'me',
 'to',
 'call',
 'john',
 'at',
 '3',
 'pm',
 'search',
 'for',
 'nearby',
 'pizza',
 'restaurants',
 'set',
 'the',
 'thermostat',
 'to',
 '72',
 'degrees',
 'turn',
 'off',
 'the',
 'lights',
 'in',
 'the',
 'living',
 'room',
 'what',
 's',
 'the',
 'weather',
 'like',
 'today',
 'find',
 'a',
 'recipe',
 'for',
 'lasagna',
 'send',
 'a',
 'message',
 'to',
 'sarah',
 'saying',
 'meeting',
 'is',
 'at',
 '10',
 'schedule',
 'a',
 'meeting',
 'with',
 'the',
 'team',
 'next',
 'monday',
 'at',
 '9',
 'am',
 'how',
 's',
 'the',
 'traffic',
 'to',
 '

In [35]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [38]:
n_words = 6
input_words = []
next_words = []

for i in range(len(tokens)- n_words):
    input_words.append(tokens[i:i + n_words])
    next_words.append(tokens[i + n_words])

In [42]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype =bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype= bool)

In [43]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i,unique_token_index[next_words[i]]] = 1

In [48]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [50]:
model.compile(loss = "categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])
model.fit(X,y, batch_size=128, epochs=100, shuffle=True)


Epoch 1/100
15/15 [==============================] - 2s 22ms/step - loss: 4.9073 - accuracy: 0.0582
Epoch 2/100
15/15 [==============================] - 0s 26ms/step - loss: 4.3726 - accuracy: 0.1004
Epoch 3/100
15/15 [==============================] - 0s 27ms/step - loss: 3.2051 - accuracy: 0.2146
Epoch 4/100
15/15 [==============================] - 0s 32ms/step - loss: 1.8710 - accuracy: 0.4757
Epoch 5/100
15/15 [==============================] - 0s 29ms/step - loss: 1.0650 - accuracy: 0.7064
Epoch 6/100
15/15 [==============================] - 0s 25ms/step - loss: 0.5013 - accuracy: 0.8772
Epoch 7/100
15/15 [==============================] - 0s 25ms/step - loss: 0.3483 - accuracy: 0.9082
Epoch 8/100
15/15 [==============================] - 0s 30ms/step - loss: 0.2484 - accuracy: 0.9301
Epoch 9/100
15/15 [==============================] - 0s 31ms/step - loss: 0.2584 - accuracy: 0.9263
Epoch 10/100
15/15 [==============================] - 0s 33ms/step - loss: 0.2219 - accuracy: 0.9359

Epoch 80/100
15/15 [==============================] - 0s 28ms/step - loss: 1.3929e-04 - accuracy: 1.0000
Epoch 81/100
15/15 [==============================] - 1s 39ms/step - loss: 1.3730e-04 - accuracy: 1.0000
Epoch 82/100
15/15 [==============================] - 0s 25ms/step - loss: 1.3532e-04 - accuracy: 1.0000
Epoch 83/100
15/15 [==============================] - 0s 28ms/step - loss: 1.3343e-04 - accuracy: 1.0000
Epoch 84/100
15/15 [==============================] - 0s 27ms/step - loss: 1.3158e-04 - accuracy: 1.0000
Epoch 85/100
15/15 [==============================] - 0s 26ms/step - loss: 1.2978e-04 - accuracy: 1.0000
Epoch 86/100
15/15 [==============================] - 0s 26ms/step - loss: 1.2802e-04 - accuracy: 1.0000
Epoch 87/100
15/15 [==============================] - 0s 26ms/step - loss: 1.2634e-04 - accuracy: 1.0000
Epoch 88/100
15/15 [==============================] - 0s 26ms/step - loss: 1.2471e-04 - accuracy: 1.0000
Epoch 89/100
15/15 [==============================] - 1

In [51]:
model.save("textmodel.h5")

/Users/amanmehra/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [52]:
model = load_model("textmodel.h5")

In [70]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0,i, unique_token_index[word]] = 1
    predictions = model.predict(X)[0]
    return np.argpartition(predictions, n_best)[n_best:]

In [71]:
possible = predict_next_word("Set a timer for ",5)

1/1 [==============================] - 0s 79ms/step


In [72]:
print([unique_tokens[idx] for idx in possible])

['read', 'class', 'monday', 'health', 'where', 'rate', 'are', 'ppt', 'lasagna', 'alarm', 'send', 'calendar', 'pictures', 'set', 'yoga', 'add', 'latest', 'create', 'weather', 'of', '9', 'right', 'go', 'dining', 'near', 'document', 'named', 'start', 'eclipse', 'music', 'episode', 'like', 'today', 'repository', 'lunar', 'water', 'spa', 'modified', 'more', 'two', 'front', '25', 'dim', 'open', 'order', 'bali', 'timer', 'it', 'joke', 'tech', 'in', 'is', 'this', 'italian', 'team', 'time', 'table', 'healthy', 'good', 'system', 'stats', 'git', 'french', 'list', 'living', 'lock', 'flight', 'spanish', 'me', 'top', 'message', 'minutes', 'files', 'file', 'faucet', 'some', 'movies', 'much', 'email', 'up', 'door', 'video', 'nearby', 'news', 'next', 'day', 'daily', 'smoothie', 'show', 'coffee', 'check', 'changes', 'phone', 'call', 'pizza', 'plants', 'when', 'battery', 'security', 'podcast', 'pods', 'at', 'search', 'schedule', 'quarterly', 'an', 'workout', '72', 'remind', 'report', '7', 'year', 'run', 

In [74]:
def generate_text(input_text,text_length = 1, creativity=5):
    word_sequence = input_text.split()
    current = 0
    for _ in range(text_length):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current +=1
    return " ".join(word_sequence)

In [100]:
generate_text("jarvis, tell me",1)

'jarvis, tell me much'